In [ ]:
#필요한 라이브러리 호출 공간
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

데이터 전처리

In [ ]:
# 데이터 불러오기
file_name = ""
def reading_csv(filename):
    df = pd.read_csv(filename)
    print(df.shape)
    print(df.head())
    return df

df = reading_csv(file_name)

# 결측치 확인
print(df.isnull().sum())

# 결측치 처리 (평균 or 최빈값 or 중위값)
si = SimpleImputer(strategy="mean")
ss = StandardScaler()
le = LabelEncoder()
oe = OneHotEncoder(sparse=False)
num_col = []
cat_col = []

def processing(df):
    df = df.copy()

    # 수치형 컬럼 처리
    df[num_col] = si.fit_transform(df[num_col])
    df['파생변수'] = pd.cut(df['col'], bins=[0, 18, 30, 50, 100], labels=['Teen', 'Young', 'Adults', 'Senior'])
    df[num_col] = ss.fit_transform(df[num_col])

    # 범주형 컬럼 처리
    for col in cat_col:# label encoding
            df[col] = le.fit_transform(df[col])

    return df

processed_df = processing(df)
print(processed_df.head())


Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# 타겟 변수 지정
target_col = 'target'  # 실제 변수명으로 변경

X = df.drop(columns=[target_col])
y = df[target_col]

# 훈련/검증 데이터 분할 (8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


모델 적용

In [ ]:
# 모델 적용 및 성능 평가
from sklearn import linear_model  
# 로지스틱 회귀 (이진 분류에 특화)
from sklearn.linear_model import LogisticRegression  
# 앙상블 모델: 여러 결정트리를 결합하여 성능 향상
from sklearn.ensemble import RandomForestClassifier  
# 단순 신경망 기반의 선형 분류기
from sklearn.linear_model import Perceptron  
# 확률적 경사하강법 기반 선형 분류기
from sklearn.linear_model import SGDClassifier  
# 결정 트리 기반 분류기 (if-else 형태의 트리 구조)
from sklearn.tree import DecisionTreeClassifier  
# K-최근접 이웃 알고리즘 (거리 기반 분류)
from sklearn.neighbors import KNeighborsClassifier  
# 서포트 벡터 머신 (복잡한 경계 분류, 마진 기반)
# LinearSVC: 선형 SVM (SVM의 일종, 대용량에 빠르고 선형 분리에 적합)
from sklearn.svm import SVC, LinearSVC  
# 확률 기반 분류기 (조건부 독립 가정)
from sklearn.naive_bayes import GaussianNB  

# 🔹 [3] 시각화 도구 임포트
import seaborn as sns  
%matplotlib inline  
from matplotlib import pyplot as plt  
from matplotlib import style  # 그래프 스타일 지정용

# 의사결정 나무 --> 복자도가 높아지면 일반화 성능이 떨어짐, 과적합 가능성
decision_tree=DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred=decision_tree.predict(X_test)
train_acc_decision_tree=round(decision_tree.score(X_train, y_train)*100, 2)
test_acc_decision_tree=round(decision_tree.score(X_test, y_test)*100, 2)
train_acc_decision_tree,test_acc_decision_tree


# 랜덤 포레스트 --> 과적합 방지에 강함, 하지만 하이퍼파라미터 기본값 사용 등으로 일반화 성능이 크게 향상되지 않음
# max_depth, min_sample_leaf 제한
# n_estimator 증가
# GridSearchCV 사용 --> 최적의 하이퍼 파라미터 조건 탐색
random_forest=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_prediction=random_forest.predict(X_test)
random_forest.score(X_train, y_train)
train_acc_random_forest=round(random_forest.score(X_train, y_train)*100, 2)
test_acc_random_forest=round(random_forest.score(X_test, y_test)*100, 2)
train_acc_random_forest,test_acc_random_forest


# 로지스틱 회귀 --> 과적합 위험 낮음, 특히 특성이 별로 없을 때
log_reg=LogisticRegression()
log_reg.fit(X_train, y_train)
Y_pred=log_reg.predict(X_test)
train_acc_log=round(log_reg.score(X_train, y_train)*100, 2)
test_acc_log=round(log_reg.score(X_test, y_test)*100, 2)
train_acc_log, test_acc_log


# 나이브 베이즈 --> 로지스틱 회귀보다 성능 낮음
gaussian=GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred=gaussian.predict(X_test)
train_acc_gaussian=round(gaussian.score(X_train, y_train)*100, 2)
test_acc_gaussian=round(gaussian.score(X_test, y_test)*100, 2)
train_acc_gaussian,test_acc_gaussian

# Linear SVC
svc=LinearSVC()
svc.fit(X_train, y_train)
Y_pred=svc.predict(X_test)
train_acc_svc=round(svc.score(X_train, y_train)*100, 2)
test_acc_svc=round(svc.score(X_test, y_test)*100, 2)
train_acc_svc,test_acc_svc

# Perceptron
perceptron=Perceptron(max_iter=5)
perceptron.fit(X_train, y_train)
Y_pred=perceptron.predict(X_test)
train_acc_perceptron=round(perceptron.score(X_train, y_train)*100, 2)
test_acc_perceptron=round(perceptron.score(X_test, y_test)*100, 2)
train_acc_perceptron,test_acc_perceptron


#SDGclassifier --> 과소적합 (max_iter 늘려야됨)
sgd=linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
Y_pred=sgd.predict(X_test)
sgd.score(X_train, y_train)
train_acc_sgd=round(sgd.score(X_train, y_train)*100, 2)
test_acc_sgd=round(sgd.score(X_test, y_test)*100, 2)
train_acc_sgd,test_acc_sgd


#모델 비교 방식
results = pd.DataFrame({
 'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest','Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 'Decision Tree'],
 'train_Score': [train_acc_svc, train_acc_knn, train_acc_log,train_acc_random_forest,
train_acc_gaussian, train_acc_perceptron, train_acc_sgd, train_acc_decision_tree],
 'test_Score': [test_acc_svc, test_acc_knn,test_acc_log, test_acc_random_forest, test_acc_gaussian, test_acc_perceptron, test_acc_sgd, test_acc_decision_tree]})
result_df=results.sort_values(by='train_Score', ascending=False)
result_df=result_df.set_index('Model')
result_df.head(10)
